In [1]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.chains import LLMChain

In [3]:
llm = Ollama(model="llama3.1")
llm

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_24228\2895630186.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


Ollama(model='llama3.1')

In [2]:
train = pd.read_csv('demand-forecasting-kernels-only/train.csv', parse_dates=['date'])
test = pd.read_csv('demand-forecasting-kernels-only/test.csv', parse_dates=['date'])

In [4]:
# Define the prompt template
demo_template = """
You are a data analyst. You are provided with a dataset about supply chain information.

Here is the dataset structure:
{train}

Please analyze the data and provide insights about:
1. Key trends and patterns in the supply chain (e.g., delays, high-demand items, etc.).
2. Any anomalies or outliers in the data.
3. Suggestions for visualization to better understand the data (e.g., bar charts, line graphs, or heatmaps).

Respond with a detailed explanation and recommendations.
"""

# Create the prompt
prompt_template = PromptTemplate(
    input_variables=["train"],
    template=demo_template
)

from langchain.chains import LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

response = chain.run({"train": train})

response

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_24228\2349737343.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_24228\2349737343.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"train": train})


"**Data Analysis and Insights**\n\nAfter analyzing the supply chain dataset, I've identified several key trends and patterns, as well as anomalies and outliers.\n\n**1. Key Trends and Patterns:**\n\n* **Sales Volume:** The data shows a consistent sales volume across different items and stores over the years. However, there are fluctuations in sales volume throughout the year, with higher sales during the holiday season (December).\n* **Store Performance:** Store 10 consistently has higher sales than other stores, indicating that it may be a high-demand store or have better inventory management.\n* **Item Demand:** Item 50 is consistently among the top-selling items across all stores, suggesting that it is in high demand. Conversely, item 1 shows relatively low sales throughout the year.\n\n**2. Anomalies and Outliers:**\n\n* **Anomalous Sales Spike:** On December 29th, 2017 (row 912997), there was an unexpected spike in sales for store 10 with item 50, which deviates from the overall t

In [5]:
# Define the prompt template
demo_template = """
I want you to explain to me what data is {train},
explain each column, what it represents and how to use this data in the Time series Forecasting project
"""

# Create the prompt
prompt_template = PromptTemplate(
    input_variables=["train"],
    template=demo_template
)

from langchain.chains import LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

response = chain.run({"train": train})

response

"Let's break down each column of your dataset:\n\n**1. date**\n\n* This column represents the date of sales.\n* It contains a timestamp for every sale made, ranging from January 2013 to December 2017 (inclusive).\n* The format is `YYYY-MM-DD`, which is a standard way to represent dates in many programming languages and databases.\n\nExample: `2013-01-01` means January 1st, 2013.\n\n**2. store**\n\n* This column represents the ID of the store where each sale was made.\n* In this dataset, there's only one store with an ID of `10`.\n* If you were to use this data in a real-world scenario, you'd likely have multiple stores with unique IDs, and this column would contain those IDs.\n\nExample: `10` is the ID of the single store in this dataset.\n\n**3. item**\n\n* This column represents the ID of the item that was sold.\n* In this dataset, there's only one item with an ID of `50`.\n* Similar to the `store` column, if you were using this data in a real-world scenario, you'd likely have multip

In [11]:
# Analysis Data
def analysis_data(train, llm):
    # Extract dataset information
    try:
        data_info = train.describe().to_string()
    except Exception as e:
        raise ValueError("Unable to extract dataset information. Ensure 'train' is a valid DataFrame.") from e

    # Define the prompt for analysis
    analysis_prompt = '''
    You are a data analyst. You are provided with a dataset described as follows:
    {data_info}

    Please analyze the data and provide insights about:
    1. Key trends and patterns in the data.
    2. Any anomalies or outliers in the data.
    3. Recommendations or actionable insights based on the analyzed data.
    '''

    # Define the prompt template
    analysis_template = PromptTemplate(
        input_variables=["data_info"],
        template=analysis_prompt
    )

    # Create a chain for analysis
    analysis_chain = LLMChain(llm=llm, prompt=analysis_template)

    # Run the chain on the data
    analysis = analysis_chain.run(data_info=data_info)

    # Return the analysis
    return analysis

# Example usage:
result = analysis_data(train=train, llm=llm)
print("Analysis Data:\n", result)


Analysis Data:
 Based on the provided dataset, I've performed a thorough analysis to extract key trends, identify anomalies, and provide actionable insights.

**Key Trends and Patterns:**

1. **Increasing Sales Over Time**: The mean sales value has increased from 0 in 2013 to 52.25 in the latest available data point (2015-07-02). This suggests a positive trend in sales over time.
2. **Store Count Consistency**: The count of stores remains relatively consistent at around 913,000 for all dates, indicating that the number of stores hasn't changed significantly over the analysis period.
3. **Item Variety and Frequency**: There are 10 unique store values (1-10), with a mean value of 5.5, suggesting a moderate number of items sold across different stores. The item '25.500000' appears most frequently, implying it's a popular product.
4. **Skewed Distribution**: The standard deviation for sales is relatively high (28.8), indicating a skewed distribution. This suggests that some stores or produ

In [14]:
def analysis_data(train, llm):
    # Extract dataset information
    try:
        data_info = train.describe().to_string()
    except Exception as e:
        raise ValueError("Unable to extract dataset information. Ensure 'train' is a valid DataFrame.") from e

    # Define the prompt for analysis
    analysis_prompt = '''
    explain this line 
    {data_info}
    '''

    # Define the prompt template
    analysis_template = PromptTemplate(
        input_variables=["data_info"],
        template=analysis_prompt
    )

    # Create a chain for analysis
    analysis_chain = LLMChain(llm=llm, prompt=analysis_template)

    # Run the chain on the data
    analysis = analysis_chain.run(data_info=data_info)

    # Return the analysis
    return analysis

# Example usage:
result = analysis_data(train=train, llm=llm)
print("Analysis Data:\n", result)


Analysis Data:
 This line of text appears to be output from a statistical analysis or summary of data, likely generated by a programming language such as Python using libraries like Pandas for data manipulation and NumPy for numerical computations.

Here's a breakdown of what each part of this line means:

1. **date**: This seems to represent the date at which the data was being summarized or analyzed. The output shows several dates: `2015-07-02 11:59:59.999999744`, `2014-04-02 00:00:00`, `2016-10-01 00:00:000`, and `2017-12-31 00:00:00`. These are likely dates related to the data or the analysis itself, possibly representing different quantiles (percentile values) such as min (minimum), max (maximum), 25%, 50% (median), and 75%.

2. **store**, **item**, and **sales** columns:
   - **count**: The total count of records is `913000`.
   - **mean**: These are the mean values for each column, indicating that on average there were about 5.5 stores, 25.5 items, and $52.25 in sales per unit o